In [9]:
import warnings 
warnings.filterwarnings('ignore')

In [10]:
import pandas as pd

train_data = pd.read_json("/kaggle/input/datasetner/train.jsonl", lines=True)
test_data = pd.read_json("/kaggle/input/datasetner/test.jsonl", lines=True)

In [11]:
train_data.head()

,ners,sentences,id
0,"[[0, 5, CITY], [16, 23, PERSON], [34, 41, PERS...",Бостон взорвали Тамерлан и Джохар Царнаевы из ...,0
1,"[[21, 28, PROFESSION], [53, 67, ORGANIZATION],...",Умер избитый до комы гитарист и сооснователь г...,1
2,"[[0, 4, PERSON], [37, 42, COUNTRY], [47, 76, O...",Путин подписал распоряжение о выходе России из...,2
3,"[[0, 11, PERSON], [36, 47, PROFESSION], [49, 6...",Бенедикт XVI носил кардиостимулятор\nПапа Римс...,3
4,"[[0, 4, PERSON], [17, 29, ORGANIZATION], [48, ...",Обама назначит в Верховный суд латиноамериканк...,4


In [12]:
test_data = test_data.rename(columns={"senences": "sentences"})
test_data.head()

,sentences,id
0,Владелец «Бирмингема» получил шесть лет тюрьмы...,584
1,Акция протеста на Майдане Независимости объявл...,585
2,Фольксваген может перейти под контроль Порше \...,586
3,В Москве покажут фильмы Чарли Чаплина с живой ...,587
4,Чулпан Хаматова сыграет главную роль в фильме ...,588


In [14]:
from datasets import load_dataset

ent_types = load_dataset('MalakhovIlya/RuNNE', 'ent_types')['ent_types']
ent_types = ent_types['type']
list(ent_types)

['AGE',
 'AWARD',
 'CITY',
 'COUNTRY',
 'CRIME',
 'DATE',
 'DISEASE',
 'DISTRICT',
 'EVENT',
 'FACILITY',
 'FAMILY',
 'IDEOLOGY',
 'LANGUAGE',
 'LAW',
 'LOCATION',
 'MONEY',
 'NATIONALITY',
 'NUMBER',
 'ORDINAL',
 'ORGANIZATION',
 'PENALTY',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'PROFESSION',
 'RELIGION',
 'STATE_OR_PROVINCE',
 'TIME',
 'WORK_OF_ART']

# Spacy solution

In [15]:
def sort_entities(entities):
    """
    Sort entities by their length in descending order to prioritize larger entities.

    This function ensures that larger entities are considered first, which can be 
    useful when larger contexts carry more specific or relevant information.

    Args:
        entities (list of tuples): A list of tuples where each tuple represents 
                                   an entity defined as (start_index, end_index, label).

    Returns:
        list of tuples: The sorted list of entities, with longer entities appearing first.
    """
    return sorted(entities, key=lambda x: x[1] - x[0], reverse=True)

def remove_overlapping_entities(entities):
    """
    Remove overlapping entities from a list, preferring to keep the longest entity
    in cases where overlaps occur.

    This function is particularly useful in named entity recognition tasks where 
    overlapping entities can be identified by the model, and a decision needs to be
    made to keep the most relevant (typically the longest) entity.

    Args:
        entities (list of tuples): A list of tuples representing the entities. 
                                   Each tuple is formatted as (start_index, end_index, label).

    Returns:
        list of tuples: A list of entities with overlaps removed. Each entity is 
                        extended to include its end_index by adding 1 to ensure inclusive indexing.
    """
    
    span_set = set()  # Tracks covered indices
    non_overlapping_entities = []

    for start, end, label in entities:
        if not any(index in span_set for index in range(start, end + 1)):
            non_overlapping_entities.append((start, end + 1, label))  # +1 to make the end index inclusive
            span_set.update(range(start, end + 1))

    return non_overlapping_entities


# Prepare the training data for spaCy with sorted and non-overlapping entities
prepared_data = [
    (row['sentences'], {"entities": remove_overlapping_entities(sort_entities(row['ners']))})
    for _, row in train_data.iterrows()
]

In [16]:
!python3 -m spacy download ru_core_news_lg -q

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [17]:
import random
import spacy
from spacy.training import Example
import ru_core_news_lg

nlp = ru_core_news_lg.load()
optimizer = nlp.initialize()
num_iterations = 10

for iteration in range(num_iterations):
    random.shuffle(prepared_data)  # Shuffle training data each epoch

    losses = {}
    for idx, (text, annotations) in enumerate(prepared_data):
        doc = nlp.make_doc(text)
        training_example = Example.from_dict(doc, annotations)
        nlp.update([training_example], sgd=optimizer, losses=losses)

    print(f'Epoch {iteration + 1}/{num_iterations}, Loss: {losses.get("ner", 0):.4f}')

[2024-04-28 13:23:55,237] [INFO] Created vocabulary
[2024-04-28 13:23:55,239] [INFO] Finished initializing nlp object


Epoch 1/10, Loss: 34955.8723
Epoch 2/10, Loss: 20337.4654
Epoch 3/10, Loss: 15832.1105
Epoch 4/10, Loss: 12428.6963
Epoch 5/10, Loss: 9994.4448
Epoch 6/10, Loss: 8044.7176
Epoch 7/10, Loss: 6832.7917
Epoch 8/10, Loss: 5691.2122
Epoch 9/10, Loss: 4779.8625
Epoch 10/10, Loss: 4520.9024


In [18]:
import json

# Process test data to extract named entities and prepare for serialization
extracted_entities = []
for index, record in test_data.iterrows():
    document = nlp(record['sentences'])
    entities = [{'start': ent.start_char, 'end': ent.end_char - 1, 'type': ent.label_}
                for ent in document.ents if ent.label_ in ent_types]
    
    extracted_entities.append({'id': record['id'], 'entities': entities})

# Write the extracted entities to a JSONL file
with open("test.jsonl", "w", encoding="utf-8") as file:
    for data in extracted_entities:
        file.write(json.dumps(data) + "\n")

/opt/conda/lib/python3.10/site-packages/spacy/pipeline/attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
